# MC Dropout

In [1]:
%load_ext autoreload

In [2]:
import os, sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import  Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten,Conv2D, MaxPooling2D, Input
from tensorflow.keras.datasets import mnist
import tensorflow.keras.backend as K

In [3]:
BASE_PATH = os.path.join(os.getcwd(), "..", "..")
MODULES_PATH = os.path.join(BASE_PATH, "modules")

sys.path.append(MODULES_PATH)
from models import setup_growth

In [4]:
setup_growth()

1 Physical GPU's,  1 Logical GPU's


In [5]:
batch_size = 128
num_classes = 10
epochs = 12

# Input image dimensions
img_rows, img_cols = 28, 28

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [8]:
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255

print("x_train shape: ", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

x_train shape:  (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [9]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [10]:
def get_dropout(input_tensor, p=0.5, mc=False):
    if mc:
        return Dropout(p)(input_tensor, training=True)
    else:
        return Dropout(p)(input_tensor)


def get_model(mc=False, act="relu"):
    inp = Input(input_shape)
    x = Conv2D(32, kernel_size=(3, 3), activation=act)(inp)
    x = Conv2D(64, kernel_size=(3, 3), activation=act)(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = get_dropout(x, p=0.25, mc=mc)
    x = Flatten()(x)
    x = Dense(128, activation=act)(x)
    x = get_dropout(x, p=0.5, mc=mc)
    out = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=inp, outputs=out)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model


In [11]:
model = get_model()
mc_model = get_model(mc=True)

In [12]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=10, verbose=1, validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 16s 34ms/step - loss: 2.2863 - accuracy: 0.1314 - val_loss: 2.2512 - val_accuracy: 0.2841
Epoch 2/10
469/469 [==============================] - 3s 7ms/step - loss: 2.2284 - accuracy: 0.2510 - val_loss: 2.1809 - val_accuracy: 0.5045
Epoch 3/10
469/469 [==============================] - 4s 8ms/step - loss: 2.1565 - accuracy: 0.3565 - val_loss: 2.0896 - val_accuracy: 0.6001
Epoch 4/10
469/469 [==============================] - 4s 8ms/step - loss: 2.0578 - accuracy: 0.4449 - val_loss: 1.9655 - val_accuracy: 0.6776
Epoch 5/10
469/469 [==============================] - 4s 8ms/step - loss: 1.9308 - accuracy: 0.5121 - val_loss: 1.8039 - val_accuracy: 0.7329
Epoch 6/10
469/469 [==============================] - 3s 7ms/step - loss: 1.7728 - accuracy: 0.5623 - val_loss: 1.6110 - val_accuracy: 0.7615
Epoch 7/10
469/469 [==============================] - 3s 7ms/step - loss: 1.5987 - accuracy: 0.6078 - val_loss: 1.4055 - val_accuracy: 0.7871
Epoc

In [13]:
score = model.evaluate(x_test, y_test, verbose=0)

print("Test loss: ", score[0])
print("Test accuracy: ", score[1])

Test loss:  0.9142003059387207
Test accuracy:  0.8274000287055969


In [14]:
h_mc = mc_model.fit(x_train, y_train, batch_size=batch_size, epochs=10, verbose=1, validation_data=(x_test, y_test))

Epoch 1/10
469/469 [==============================] - 4s 8ms/step - loss: 2.2776 - accuracy: 0.1641 - val_loss: 2.2510 - val_accuracy: 0.2255
Epoch 2/10
469/469 [==============================] - 4s 8ms/step - loss: 2.2221 - accuracy: 0.2821 - val_loss: 2.1882 - val_accuracy: 0.3380
Epoch 3/10
469/469 [==============================] - 4s 7ms/step - loss: 2.1501 - accuracy: 0.3892 - val_loss: 2.1030 - val_accuracy: 0.4360
Epoch 4/10
469/469 [==============================] - 3s 7ms/step - loss: 2.0516 - accuracy: 0.4639 - val_loss: 1.9864 - val_accuracy: 0.5084
Epoch 5/10
469/469 [==============================] - 3s 7ms/step - loss: 1.9173 - accuracy: 0.5278 - val_loss: 1.8242 - val_accuracy: 0.5638
Epoch 6/10
469/469 [==============================] - 4s 8ms/step - loss: 1.7476 - accuracy: 0.5702 - val_loss: 1.6458 - val_accuracy: 0.5955
Epoch 7/10
469/469 [==============================] - 4s 8ms/step - loss: 1.5647 - accuracy: 0.6089 - val_loss: 1.4678 - val_accuracy: 0.6274
Epoch 

In [15]:
import tqdm

mc_predictions = []
for i in tqdm.tqdm(range(500)):
    y_p = mc_model.predict(x_test, batch_size=1000)
    mc_predictions.append(y_p)
    

100%|██████████| 500/500 [01:18<00:00,  6.39it/s]


In [16]:
accs = []
for y_p in mc_predictions:
    acc = accuracy_score(y_test.argmax(axis=1), y_p.argmax(axis=1))
    accs.append(acc)
    
print("MC accuracy: {:.1%}".format(sum(accs)/len(accs)))

MC accuracy: 70.0%


In [18]:
mc_predictions[0].shape

(10000, 10)

In [19]:
stacked = np.stack(mc_predictions, axis=2)
stacked.shape

(10000, 10, 500)

In [20]:
stacked.shape

(10000, 10, 500)

In [21]:
maxes = stacked.argmax(axis=1)
maxes.shape

(10000, 500)

In [22]:
stacked.argmax(axis=1).shape

(10000, 500)

In [23]:
means = np.mean(stacked, axis=2)

In [24]:
accuracy_score(y_test.argmax(axis=1), means.argmax(axis=1))

0.8258

In [25]:
mc_model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.0542 - accuracy: 0.6970


[1.0541653633117676, 0.6970000267028809]

In [26]:
batches = np.split(stacked, 500, axis=2)

In [27]:
all_accs = []
for batch in batches:
    reshaped = batch.reshape(len(batch), 10)
    new_score = accuracy_score(y_test.argmax(axis=1), reshaped.argmax(axis=1))
    all_accs.append(new_score)

In [28]:
sum(all_accs)/len(all_accs)

0.6999017999999994

# Debugg Mc Dropout

In [29]:
from active_learning import TrainConfig
from bayesian import McDropout

In [30]:
wp_mc_model = McDropout(mc_model, TrainConfig())

In [31]:
wp_mc_model.evaluate(x_test, y_test, batch_size=300)

[2.4117420833379772, 0.0]

In [32]:
results = wp_mc_model.batch_prediction(x_test, n_times=10, batch_size=1000)

In [33]:
results.shape

(10000, 10, 10)

In [35]:
batches = np.split(results, 10, axis=1)

In [36]:
accs = []
for batch in batches:
    reshaped = batch.reshape(len(batch), 10)
    score = accuracy_score(y_test.argmax(axis=1), reshaped.argmax(axis=1))
    print(score)
    accs.append(score)

0.1021
0.1026
0.1019
0.1014
0.1006
0.0977
0.1045
0.1052
0.1019
0.0956


In [114]:
sum(accs)/len(accs)

0.10246999999999999

In [115]:
g_model = wp_mc_model.get_model()

In [121]:
g_model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.0524 - accuracy: 0.7008


[1.0524163246154785, 0.7008000016212463]